In [1]:
from astropy import time
import numpy as np
import glob
import os.path
import json
import matplotlib.pyplot as plt
from functools import partial
import signal

from IPython import display
import pandas as pd
import panel as pn
pn.extension("tabulator")
import hvplot.pandas
import holoviews as hv
hv.extension("bokeh")

from astropy.coordinates import EarthLocation
from astropy.time import Time, TimeDelta
import astropy.units as u

import sched_funcs, antmon
from event import labels, lookup
from dsacalib import weights

try:
    from dsaT3.utils import archive
except ImportError:
    print("No T3. Skipping...")

try:
    from dsautils import dsa_store
    ds = dsa_store.DsaStore()
except ImportError:
    ds = None
    
trig_pane = None
sched_pane = None
calibration_pane = None
bokeh_pane = None

No T3. Skipping...


### Trigger inspection

In [2]:
base = '/operations/T3'

In [3]:
fn_pngs = sorted(glob.glob(os.path.join(base, '*png')))
fn_jsons = sorted(glob.glob(os.path.join(base, '*json')))
fn_jsons = [fn for fn in fn_jsons if 'management' not in fn]

In [4]:
triggernames = [fn[:-5] for fn in fn_jsons]

if len(fn_pngs) != len(fn_jsons):
    print('Warning: different number of pngs and jsons')
    # TODO: add logic to find list of triggernames?

print(f'Found {len(triggernames)} complete triggers')

##optional: parse management.json file to see what else is going on?
#with open('/home/ubuntu/data/T3/management.json', 'r') as fp:
#    dd = json.load(fp)

Found 263 complete triggers


In [5]:
def build_triggertab():
    if triggernames:
        dds = []
        keys = ['mjds', 'snr', 'dm', 'ibox', 'ibeam', 'trigname', 'ra', 'dec', 'probability', 'label', 'save']
        for triggername in triggernames:
            dd = labels.readfile(triggername + '.json')
            if dd == None:
                continue
            dd2 = dd.fromkeys(keys)
            for key in keys:
                if key in dd:
                    dd2[key] = dd[key]
                else:
                    dd2[key] = -1
            dds.append(dd2)
        df = pd.DataFrame.from_dict(dds)
        df.set_index('trigname', inplace=True)
        df_pane = pn.widgets.Tabulator(df[::-1], layout='fit_data_table', pagination='remote', page_size=10,
                                      selectable='checkbox', height=400, width=1000)
    else:
        df_pane = pn.widgets.Tabulator(pd.DataFrame(), layout='fit_data_table', pagination='remote', page_size=10,
                                      selectable='checkbox', height=400, width=1000)

        
    def set_label(label=False):
        if label is not None:
            if label == 'None':
                label = ''
            if len(df_pane.selection):
                for trigname in df_pane.selected_dataframe.index:
                    labels.set_label(label, candname=trigname)
                return f'Labeled {list(df_pane.selected_dataframe.index)} as {label}'

    def set_notes(note):
        if note:
            if len(df_pane.selection):
                for trigname in df_pane.selected_dataframe.index:
                    labels.set_notes(note, candname=trigname)
                return f'Note added for {list(df_pane.selected_dataframe.index)}'
        else:
            if len(df_pane.selection):
                for trigname in df_pane.selected_dataframe.index:
                    labels.set_notes('', candname=trigname)            
                return f'Note removed for {list(df_pane.selected_dataframe.index)}'

    def run_lookup(event):
        lookup_text.value = 'Looking...'
        if len(df_pane.selection):
            ss = ''
            for trigname in df_pane.selected_dataframe.index:
                ra, dec = df.loc[trigname][['ra', 'dec']]
                result = lookup.find_associations(float(ra), float(dec), nvss_radius=3.3*3600, atnf_radius=3.3*3600)
                for src in result:
                    ss += f'{trigname} is {src[2][0]} arcsec from {src[0]} source. '
            if ss:
                lookup_text.value = ss
            else:
                lookup_text.value = 'Nothing found.'
        else:
            lookup_text.value = f'Nothing selected.'

    def run_archive(event):
        with pn.param.set_values(png_column, loading=True):
            result = archive(datestring)
            if len(result):
                archive_text.value = f'Archived {",".join(result)}'
            else:
                archive_text.value = 'No files to archive'

    get_name = pn.widgets.Button(name='Load plot for selected row', width=900, button_type='primary')
    png_column = pn.Column('', width=900)

    def set_png(event):
        if len(df_pane.selection):
            triggername = df_pane.selected_dataframe.index[0]
            if len(df_pane.selection) > 1:
                print(f'Only using first selected row {triggername}')
            fn = os.path.join(base, triggername + '.png')
            if os.path.exists(fn):
                with pn.param.set_values(png_column, loading=True):
                    image = display.Image(fn)
                    _ = png_column.pop(0)
                    png_column.append(image)
            archive_text.value = f"{df_pane.selection} {fn}"
        else:
            _ = png_column.pop(0)
            png_column.append("Nothing selected")

    _ = get_name.param.watch(set_png, 'value')

    datestring = ds.get_dict('/cnf/datestring') if ds is not None else None
    archive_text = pn.widgets.TextInput(value=datestring, width=250)
    archive_button = pn.widgets.Button(name='Debug text', width=250, button_type='primary')
#    archive_button.on_click(run_archive)

    lookup_text = pn.widgets.TextInput(value='Ready', width=650)
    lookup_input = pn.widgets.Button(name='Run coordinate lookup on selected row', width=250, button_type='primary')
    lookup_input.on_click(run_lookup)
    lookup_pane = pn.Row(lookup_input, lookup_text)

    label_group = pn.widgets.RadioButtonGroup(name='Add label for selected row(s)',
                                              options=['astrophysical', 'injection', 'rfi', 'instrumental',
                                                       'unsure/noise', 'save', 'None'], button_type='success',
                                              width=500, value='None')
    get_notes = pn.widgets.TextInput(placeholder='Freeform note', value='')

    label_pane = pn.Column(label_group, pn.panel(pn.bind(set_label, label_group)))
    notes_pane = pn.Column(get_notes, pn.panel(pn.bind(set_notes, get_notes)))
    ln_pane = pn.Row(label_pane, notes_pane, sizing_mode='stretch_width')
    a_pane = pn.Row(archive_button, archive_text)

# build it all
    dashboard_title = pn.pane.Markdown('# DSA-110 Trigger Dashboard')
    header = pn.pane.Markdown('Add labels and notes for selected rows')
    trig_pane = pn.Column(dashboard_title, df_pane, header, ln_pane, lookup_pane, a_pane,
                          pn.Column(get_name, png_column))        
        
    return trig_pane

In [6]:
#trig_pane = build_triggertab()
#trig_pane.servable(title='dashboard')

### Scheduling


In [7]:
def schedule(targets=None, duration=None, delay=None):
    """ Start should be astropy time object (default is "now")
    Duration is in hours (default is 24 hours)
    Delay is in hours from now.
    """

    print(f'targets, delay, duration: {targets}, {delay}, {duration}')
    ovro = EarthLocation(lon=-118.2951 * u.deg, lat=37.2317 * u.deg, height=1222 * u.m)
    catalog = sched_funcs.read_srcs('/home/user/proj/dsa110-shell/dsa110-vis/catalog.yaml')

    if targets is not None and targets != '':
        catalog['sources'] = [src for src in catalog['sources'] if src['name'] in targets]

    if duration is None or duration == '':
        duration = 24.
    else:
        duration = float(duration)

    if delay is None or delay == '':
        delay = 0.
    else:
        delay = float(delay)

    start_time = Time.now() + TimeDelta(delay*u.hour)

    # calculate actions
    print(f'Catalog: {catalog}')
    catalog2, transit_times, max_alts, stimes, end_times, northy = sched_funcs.return_times_day(catalog, start_time, duration, ovro)
    print(f'Catalog2: {catalog2}')
    actions = sched_funcs.define_actions_simple(catalog2, transit_times, max_alts, stimes, end_times, northy, recording=True)

    # save actions
    names = [src['name'] for src in catalog2]
    transits = [t.mjd[0] for t in transit_times]

    sidereal_time = Time.now().sidereal_time('apparent', longitude=ovro.lon)
    
    fig, ax = plt.subplots(1, 1)
    for name, ttran, alt, start, stop in list(zip(names, transits, max_alts, stimes, end_times)):
        ax.plot([start, stop], [alt, alt], '-', label=name)
        ax.plot(ttran, alt, 'kx')
        ax.text(Time.now().mjd, 60, f'current sid time={sidereal_time.value:.2f}',
                horizontalalignment='center', verticalalignment='center', rotation='vertical')
        ax.legend()
        ax.set_ylim(20, 110)
        ax.set_xlim(Time.now().mjd - duration/24/10, stop+duration/24/10)
        ax.set_ylabel('Elevation (deg)')
        ax.set_xlabel(f'Time (MJD start={Time.now().mjd})')
    return fig, actions

ovro = EarthLocation(lon=-118.2951 * u.deg, lat=37.2317 * u.deg, height=1222 * u.m)
catalog = sched_funcs.read_srcs('/home/ubuntu/proj/dsa110-shell/dsa110-vis/catalog.yaml')
start_time = Time.now()
duration = 24
catalog['sources'] = [src for src in catalog['sources'] if src['name'] in ['3C196']]
catalog2, transit_times, max_alts, stimes, end_times, northy = sched_funcs.return_times_day(catalog, start_time, duration, ovro)
print(max_alts, northy)
actions = sched_funcs.define_actions_simple(catalog2, transit_times, max_alts, stimes, end_times, northy, recording=True)
from astropy.coordinates import SkyCoord, AltAz
deltas = np.linspace(0., duration, 6*60*int(duration))*u.hour
times = start_time + deltas
transit_times = []
srcnames = []
max_alts = []
northy = []
repeats = int(np.ceil(max(deltas.value)/24))
src = catalog['sources'][0]
coord=SkyCoord(ra=src['RA'], dec=src['DEC'], unit=(u.hourangle, u.deg))
i = 0
times0 = times[(deltas.value < (i+1)*24) & (i*24 < deltas.value)]
aas = coord.transform_to(AltAz(obstime=times0, location=ovro))
alts = aas.alt.value
azs = aas.az.value
tt = times0[(alts >= alts.max()) * (np.abs(azs.min()) < 10)]

plt.plot(times0.mjd, alts, '.', label='alt')
plt.plot(times0.mjd, azs, '.', label='az')
plt.legend()

In [8]:
target_text = pn.widgets.TextInput(value='3C147', name='Targets (comma-delimited)', width=200, height=50)
duration_text = pn.widgets.TextInput(value=None, name='Duration', width=50, height=50)
delay_text = pn.widgets.TextInput(value=None, name='Delay', width=50, height=50)
plot_button = pn.widgets.Button(name='Make plot', width=100, button_type='primary')
plot = pn.pane.Matplotlib(width=700, height=400)
run_button = pn.widgets.Button(name='Run actions', width=100, button_type='primary')
#interrupt_button = pn.widgets.Button(name='Interrupt actions', width=100, button_type='primary')
if os.path.exists('output_text'):
    with open('output_text', 'r') as fp:
        value = fp.read()
else:
    value = ''
output_text = pn.widgets.TextAreaInput(value=value, name='Action log', width=800, height=300)

def set_plot(event):
    if len(cat_pane.selection):
        target_text.value = ','.join(cat_pane.selected_dataframe.index)

    with pn.param.set_values(plot, loading=True):
        try:
            fig, actions = schedule(targets=target_text.value, delay=delay_text.value,
                                    duration=duration_text.value)
            np.save('actions.npy', actions)
            plot.object = fig
            output_text.value = f'Actions for plot:'
            for ln in actions:
                output_text.value+=f'\nWait until {ln["time"]}'
                output_text.value+=f'\nCommand {ln["cmd"]} {ln["val"]}'
                save_output_text(output_text)

        except:
            plot.object = None

def run_actions(event):
    fig, actions = schedule(targets=target_text.value, delay=delay_text.value,
                            duration=duration_text.value)
    np.save('actions.npy', actions)
    output_text.value = f'Running actions:'
    for ln in actions:
#        if os.path.exists('interrupt'):
#            output_text.value+='\nInterrupted'
#            os.remove('interrupt')
#            break
            
        output_text.value+=f'\n{Time.now().iso} (MJD: {Time.now().mjd}): Waiting until {ln["time"]}'
        save_output_text(output_text)
        status = sched_funcs.pause_until(ln['time'])

#        if status:
#            output_text.value+='\nInterrupted'
#            save_output_text(output_text)
#            os.remove('interrupt')
#            break

        output_text.value+=f'\n{Time.now().iso} (MJD: {Time.now().mjd}: Command {ln["cmd"]} {ln["val"]}'
        save_output_text(output_text)
        sched_funcs.exec_action(ln)

def save_output_text(output_text):
    with open('output_text', 'w') as fp:
        fp.write(output_text.value)


run_button.on_click(run_actions)
plot_button.on_click(set_plot)
#interrupt_button.on_click(interrupt)

catalog = sched_funcs.read_srcs('/home/user/proj/dsa110-shell/dsa110-vis/catalog.yaml')
df_cat = pd.DataFrame.from_dict(catalog['sources'])
df_cat.set_index('name', inplace=True)
cat_pane = pn.widgets.Tabulator(df_cat, layout='fit_data_table', pagination='remote', page_size=10,
                                   selectable='checkbox', height=400, width=800)

scheduler_title = pn.pane.Markdown('# DSA-110 Scheduler')

In [9]:
sched_pane = pn.Column(pn.Row(scheduler_title), pn.Row(target_text, delay_text, duration_text,
                                                       plot_button, run_button),#, interrupt_button),
                       pn.Row(cat_pane),
                       pn.Row(output_text),
                       pn.Row(plot))

In [10]:
#sched_pane

### Calibration

In [19]:
ls /home/user/data/webPLOTS/calibration

1229+020_2023-02-03T11:29:53_averagedweights.png
1534+015_2023-02-03T14:35:06_averagedweights.png
1604+012_2023-02-03T15:04:54_averagedweights.png
2023-02-03_0022+002.pdf
2023-02-03_1534+015.pdf
2023-02-03_1604+012.pdf
2023-02-03_2136+006.pdf
2023-02-03_2212+018.pdf
2136+006_2023-02-03T20:35:53_averagedweights.png
2212+018_2023-02-03T21:11:46_averagedweights.png
allpngs/


In [11]:
def build_calpane():
    path_calplots = '/home/user/data/webPLOTS/calibration/'
    calplots = glob.glob(path_calplots + "/*.png")
    menu_calplots = pn.widgets.MenuButton(name="Select a gain plot to display",
                                          items=[os.path.basename(cp) for cp in calplots])
    load_button = pn.widgets.Button(name='Load displayed gains', width=100, button_type='primary')

    calplot_column = pn.Column('', width=900)

    def load_png(event):
        with pn.param.set_values(calplot_column, loading=True):
            print(menu_calplots.clicked)
            image = pn.pane.PNG(os.path.join(path_calplots, menu_calplots.clicked), width=1000)
            _ = calplot_column.pop(0)
            print(len(calplot_column))
            calplot_column.append(image)

    def load_weights(event):
        os.system(f"update_bfweights.py {menu_calplots.clicked.rstrip('_averagedweights.png')}")

    menu_calplots.on_click(load_png)
    load_button.on_click(load_weights)

    calibration_title = pn.pane.Markdown('# DSA-110 Calibration')

    calibration_pane = pn.Column(pn.Row(calibration_title),
                                 pn.Row(menu_calplots, load_button),
                                 pn.Row(calplot_column))
    return calibration_pane

path_calib = '/data/dsa110/T3/calibs/'
plot_cal = pn.pane.Matplotlib(width=700, height=400)
calplot_button = pn.widgets.Button(name='Make calplot', width=100, button_type='primary')

def set_calplot(event):
    with pn.param.set_values(plot, loading=True):
        try:
            bfweights = ds.get_dict('/mon/cal/bfweights')
            if 'bfname' in bfweights:
                bfname = bfweights['bfname']
            else:
                latest = max(glob.glob(os.path.join(path_calib, 'beamformer_weights_corr03_*dat')), key=os.path.getctime)
                bfname = latest.split('/')[-1].lstrip('beamformer_weights_corr03_').rstrip('.dat')
                print(bfname)
            gains = weights.read_gains(path=path_calib, bfnames=[bfname])
            fig = weights.show_gains([bfname], gains, [0], ret=True, show=False)

            plot_cal.object = fig
        except:
            plot_cal.object = None


calplot_button.on_click(set_calplot)
calibration_title = pn.pane.Markdown('# DSA-110 Calibration')

In [12]:
#calibration_pane = build_calpane()
#calibration_pane

### Antenna status (bokeh plot)

In [13]:
def build_bokeh():
    try:
        pall, source, source2, source3 = antmon.makefig()
    except:
        pall = None
    bokeh_pane = pn.pane.Bokeh(pall)
    return bokeh_pane
# automatic updates
#bokeh_pane.servable()
#def update(source, source2, source3):
#    antmon.update(source, source2, source3)
#    bokeh_pane.param.trigger('object')
#cb = pn.state.add_periodic_callback(partial(update, source, source2, source3), 1000, timeout=5000)

In [14]:
trig_pane = pn.param.ParamFunction(build_triggertab, lazy=True)
bokeh_pane = pn.param.ParamFunction(build_bokeh, lazy=True)
calibration_pane = pn.param.ParamFunction(build_calpane, lazy=True)

In [15]:
dashboard = pn.Column(pn.Row(pn.layout.tabs.Tabs(('Triggers', trig_pane),
#                                                 ('Scheduler', sched_pane),
                                                 ('Calibration', calibration_pane),
                                                 ('Antenna Monitor', bokeh_pane),
                                                dynamic=True, active=2),
                             margin=(10, 10, 10, 10), background='#f0f0f0'))

# Potential speedup with (dynamic=True) https://panel.holoviz.org/reference/layouts/Tabs.html?highlight=tabs#dynamic

In [16]:
dashboard.servable(title='dashboard')

/home/user/proj/dsa110-shell/dsa110-vis/antmon.py:97: FutureWarning: In a future version of pandas all arguments of DataFrame.set_index except for the argument 'keys' will be keyword-only.
  df.set_index('ant_num', 0, inplace=True)
/home/user/proj/dsa110-shell/dsa110-vis/antmon.py:109: FutureWarning: In a future version of pandas all arguments of DataFrame.set_index except for the argument 'keys' will be keyword-only.
  df2.set_index('ant_num', 0, inplace=True)


Column
    [0] Row(background='#f0f0f0', margin=(10, 10, 10, 10))
        [0] Tabs(active=2, dynamic=True)
            [0] ParamFunction(function, _pane=Str, lazy=True)
            [1] ParamFunction(function, _pane=Str, lazy=True)
            [2] ParamFunction(function, _pane=Bokeh, lazy=True)

In [17]:
#np.load('actions.npy', allow_pickle=True)